In [2]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"



In [3]:
from w5.models import Company


In [4]:
Company.objects.create(name="Company AAA", num_employees=120, num_chairs=150, num_tables=60)
Company.objects.create(name="Company BBB", num_employees=50, num_chairs=30, num_tables=20)
Company.objects.create(name="Company CCC", num_employees=100, num_chairs=40, num_tables=40)

<Company: Company CCC>

In [9]:
from django.db.models import *
from django.db.models.functions import *
from django.db.models.lookups import *

# class Company(models.Model):
#     name = models.CharField(max_length=100)
#     ticker = models.CharField(max_length=20, null=True)
#     num_employees = models.IntegerField()
#     num_tables = models.IntegerField()
#     num_chairs = models.IntegerField()

company = Company.objects.create(name="Google", 
                                 ticker=Upper(Value("goog")), 
                                  num_employees = 10, 
                                    num_tables=20, 
                                    num_chairs=20)
                                

In [10]:
companies = Company.objects.all().annotate(name_UPPER=Upper('name'))

In [11]:
for i in companies:
    print("NAME :"+i.name_UPPER)

NAME :COMPANY AAA
NAME :COMPANY BBB
NAME :COMPANY CCC
NAME :GOOGLE


In [18]:
company = (
   Company.objects.filter(num_employees__gt=F("num_chairs"))
   .annotate(chairs_needed=F("num_employees") - F("num_chairs"))
)

In [19]:
for i in company:
    print(i.chairs_needed)

20
60


In [16]:
#อยากได้บริษัทที่ขาดเก้าอี้ > 30
company = (
   Company.objects.filter(num_employees__gt=F("num_chairs"))
   .annotate(chairs_needed=F("num_employees") - F("num_chairs")).filter(chairs_needed__gt=30)
)

print(company[0].chairs_needed)

60


In [22]:
com1 = Company.objects.filter(GreaterThan(F("num_employees"), F("num_chairs")))

com2 = Company.objects.filter(num_employees__gt = F("num_chairs"))

#ทั้ง 2 มีค่าเท่ากัน

print(com2[0].)

<QuerySet [<Company: Company BBB>, <Company: Company CCC>]>

In [24]:
com3 = Company.objects.annotate(
...     need_chairs=GreaterThan(F("num_employees"), F("num_chairs")),
... )

print(com3[0].need_chairs)

False


In [3]:
from books.models import *

In [4]:
Book.objects.count()

59

In [8]:
from django.db.models import *
from django.db.models.functions import *
from django.db.models.lookups import *

# from django.db import models


# class Author(models.Model):
#     name = models.CharField(max_length=100)
#     age = models.IntegerField()


# class Publisher(models.Model):
#     name = models.CharField(max_length=300)


# class Book(models.Model):
#     name = models.CharField(max_length=300)
#     pages = models.IntegerField()
#     price = models.DecimalField(max_digits=10, decimal_places=2)
#     rating = models.FloatField()
#     authors = models.ManyToManyField(Author)
#     publisher = models.ForeignKey(Publisher, on_delete=models.CASCADE)
#     pubdate = models.DateField()


# class Store(models.Model):
#     name = models.CharField(max_length=300)
#     books = models.ManyToManyField(Book)

Book.objects.count()
Book.objects.aggregate(sum_price = Sum("price"))

{'sum_price': Decimal('572.41')}

In [7]:
Book.objects.aggregate(avg_rating = Avg("rating"))

{'avg_rating': 4.277966101694915}

In [21]:

pen = Book.objects.filter(publisher__name = "Penguin Books").aggregate(avg_rating = Avg("rating"))

print(pen)

{'avg_rating': 4.340000000000001}


In [24]:
ox = Book.objects.filter(publisher__name = "Oxford University Press").aggregate(avg_rating = Avg("rating"))
print(ox) 

{'avg_rating': 4.246153846153847}


In [25]:
# How to aggregate group by คือการใช้ annotate ร่วมกับ aggregate

Publisher.objects.annotate(num_books=Count("book"))



<QuerySet [<Publisher: Publisher object (1)>, <Publisher: Publisher object (2)>]>

In [30]:
Publisher.objects.annotate(avg_pubs=Avg("book__rating")).values()

<QuerySet [{'id': 1, 'name': 'Penguin Books', 'avg_pubs': 4.340000000000001}, {'id': 2, 'name': 'Oxford University Press', 'avg_pubs': 4.246153846153847}]>

In [33]:
# class Author(models.Model):
#     name = models.CharField(max_length=100)
#     age = models.IntegerField()


# class Publisher(models.Model):
#     name = models.CharField(max_length=300)


# class Book(models.Model):
#     name = models.CharField(max_length=300)
#     pages = models.IntegerField()
#     price = models.DecimalField(max_digits=10, decimal_places=2)
#     rating = models.FloatField()
#     authors = models.ManyToManyField(Author)
#     publisher = models.ForeignKey(Publisher, on_delete=models.CASCADE)
#     pubdate = models.DateField()


# class Store(models.Model):
#     name = models.CharField(max_length=300)
#     books = models.ManyToManyField(Book)


# EX group by
# หาค่าเฉลี่ย rating ของนักเขียน โดยมี rating > 4.2

Author.objects.annotate(avg_rating = Avg("book__rating")).filter(avg_rating__gt = 4.2)

<QuerySet [<Author: Author object (22)>, <Author: Author object (4)>, <Author: Author object (6)>, <Author: Author object (2)>, <Author: Author object (7)>, <Author: Author object (20)>, <Author: Author object (1)>, <Author: Author object (18)>, <Author: Author object (11)>, <Author: Author object (9)>, <Author: Author object (16)>, <Author: Author object (12)>]>